In [58]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [59]:
FeatureSetB = pd.read_csv('MIMIC_Windows/FSB_10Ws.csv')
FeatureMortality = pd.read_csv('MIMIC_Windows/FSB_Mortality.csv')
numberOfWindows = 10
numberOfFeatures = 20

In [60]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(Subject_id)

In [61]:
X = FeatureMortality['subject_id'].equals(Subject_id)

In [62]:
FeatureSet = FeatureSetB.drop(['subject_id'], axis=1)

In [63]:
FeatureSet = FeatureSet.to_numpy()

In [64]:
Labels = pd.DataFrame()
Labels['Expired'] = FeatureMortality['Expired'] 
y_values = Labels.to_numpy()
y_values = y_values.flatten()

In [65]:
nClients = NumSubjects
featureArray = FeatureSet

Testing LSTM

In [66]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import metrics
from keras.layers import LSTM
import tensorflow as tf
import sys
mod=sys.modules[__name__]

In [67]:
PR = [
      keras.metrics.AUC(name='prc',multi_label=False,curve='PR'),
    ]

In [68]:
np.random.seed(1)
tf.random.set_seed(1234)  # Fixing the seed values for reproduciblity

# Model Archieteture
model = Sequential()
model.add(LSTM(100, input_shape=(numberOfWindows,numberOfFeatures)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
opt = keras.optimizers.RMSprop(learning_rate=0.001) 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=PR)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100)               48400     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               10100     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 58,601
Trainable params: 58,601
Non-trainable params: 0
_________________________________________________________________


In [69]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
acc_score = []
re_score = []
pre_score = []
f_score = []
auroc = []
auprc = []

import numpy as np
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import TomekLinks

In [70]:
import warnings
warnings.filterwarnings("ignore")

In [71]:
import numpy as np
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

resample = NeighbourhoodCleaningRule() 
Nresample = TomekLinks()

X = featureArray
y = y_values

k = 5
ep = 75


scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
Train_shape = X.shape
X = X.reshape(int(Train_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
# enumerate the splits and summarize the distributions

for train_ix, val_ix in skf.split(X,y):
    # select rows
    X_train, X_val = X[train_ix], X[val_ix]
    y_train, y_val = y[train_ix], y[val_ix]
    # summarize train and test composition

    resample.fit_resample(X_train[:,:,0], y_train)
    X_train = X_train[resample.sample_indices_]
    y_train = y_train[resample.sample_indices_]

    history4 = model.fit(X_train,y_train, epochs = ep,batch_size=64)
    y_pred = model.predict(X_val)
    y_pred = list(map(lambda x: 0 if x<0.5 else 1, y_pred))
    #class_labels = np.argmax(y_pred, axis=1)
    y_actu = pd.Series(y_val)
    y_pred = pd.Series(y_pred)
     
    sensitivity = recall_score(y_actu, y_pred,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,pos_label = 1, average='binary')
    f1_value = f1_score(y_actu, y_pred,pos_label = 1, average='binary')
    prc = average_precision_score(y_actu, y_pred,pos_label = 1)
    roc = roc_auc_score(y_actu, y_pred)
    accuracy = accuracy_score(y_actu, y_pred)
    
    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    auroc.append(roc)
    auprc.append(prc)
   
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_roc_score = sum(auroc)/k
avg_prc_score = sum(auprc)/k


sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score
auroc = avg_roc_score
auprc = avg_prc_score

Epoch 1/100


2022-09-02 08:11:42.928091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 08:11:43.179639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 08:11:43.356050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 3s 21ms/step - loss: 0.2602 - prc: 0.3409
Epoch 2/100
72/72 [==============================] - 1s 19ms/step - loss: 0.2112 - prc: 0.5013
Epoch 3/100
72/72 [==============================] - 1s 19ms/step - loss: 0.1963 - prc: 0.5571
Epoch 4/100
72/72 [==============================] - 1s 17ms/step - loss: 0.1837 - prc: 0.6099
Epoch 5/100
72/72 [==============================] - 1s 17ms/step - loss: 0.1728 - prc: 0.6497
Epoch 6/100
72/72 [==============================] - 1s 18ms/step - loss: 0.1657 - prc: 0.6689
Epoch 7/100
72/72 [==============================] - 1s 18ms/step - loss: 0.1552 - prc: 0.7093
Epoch 8/100
72/72 [==============================] - 1s 18ms/step - loss: 0.1494 - prc: 0.7271
Epoch 9/100
72/72 [==============================] - 1s 17ms/step - loss: 0.1355 - prc: 0.7707
Epoch 10/100
72/72 [==============================] - 1s 17ms/step - loss: 0.1290 - prc: 0.7881
Epoch 11/100
72/72 [==============================] - 1s 17ms

2022-09-02 08:13:50.988643: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-02 08:13:51.048674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36/36 [==============================] - 0s 5ms/step
Epoch 1/100
72/72 [==============================] - 1s 17ms/step - loss: 0.2940 - prc: 0.7930
Epoch 2/100
72/72 [==============================] - 1s 17ms/step - loss: 0.1119 - prc: 0.8998
Epoch 3/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0634 - prc: 0.9383
Epoch 4/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0463 - prc: 0.9648
Epoch 5/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0414 - prc: 0.9674
Epoch 6/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0286 - prc: 0.9812
Epoch 7/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0241 - prc: 0.9849
Epoch 8/100
72/72 [==============================] - 1s 18ms/step - loss: 0.0193 - prc: 0.9912
Epoch 9/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0175 - prc: 0.9917
Epoch 10/100
72/72 [==============================] - 1s 17ms/step - loss: 0.0171 - prc: 0.9

In [72]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('AUROC of each fold - {}'.format(auroc))
print('Avg AUROC : {}'.format(avg_roc_score))
print('AUPRC of each fold - {}'.format(auprc))
print('Avg AUPRC : {}'.format(avg_prc_score)) 

accuracy of each fold - [0.8977272727272727, 0.958916083916084, 0.9711538461538461, 0.9842657342657343, 0.984251968503937]
Avg accuracy : 0.9592629811133747
Recall of each fold - [0.3300970873786408, 0.7184466019417476, 0.7884615384615384, 0.875, 0.8737864077669902]
Avg Reccall : 0.7171583271097834
Precision of each fold - [0.4146341463414634, 0.8043478260869565, 0.8817204301075269, 0.9479166666666666, 0.9473684210526315]
Avg Precision : 0.799197498051049
F1_Score of each fold - [0.3675675675675676, 0.7589743589743588, 0.83248730964467, 0.91, 0.909090909090909]
Avg F1_score : 0.755624029055501
AUROC of each fold - 0.8502198751487733
Avg AUROC : 0.8502198751487733
AUPRC of each fold - 0.6389620371280506
Avg AUPRC : 0.6389620371280506


In [73]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','AUROC','AUPRC','NumberOfWindows','Epochs']
dict_data = [{'model-type':'LSTM', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,"Epochs":ep,'AUROC':auroc,'AUPRC':auprc}]
metric_file = "Results/Results_LSTM.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")